In [45]:
import pandas as pd
import string, re
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [15]:
df = pd.read_excel("/media/cattiaux/DATA/Wassati/team_data/schneider/OEM_Verbatims_base.xlsx", engine='openpyxl')
# df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/OEM_Verbatims_allComment.csv")
df_labelled = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})

/tmp/ipykernel_15528/287890023.py:3: DtypeWarning: Columns (10,11,15,19,20,21,23,34,35,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labelled = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})


In [41]:
########### Global Info 

nb_scores = df["Overall Satisfaction"].count()
nb_recommend_scores = df["Likelihood to Recommend (SE)"].count()
nb_comments = len(df_labelled)
pct_comment_by_score = nb_comments/nb_scores*100
pct_recommendScore_by_score = nb_recommend_scores/nb_scores*100

pct_recommendScore_by_score

34.60742300987798

In [50]:
# Count the frequency of each score
score_counts = df['Overall Satisfaction'].value_counts().sort_index()
recommend_score_counts = df['Overall Satisfaction'].value_counts().sort_index()

# Create a bar chart
fig = go.Figure(data=go.Bar(x=score_counts.index, y=score_counts.values))
# fig = go.Figure(data=go.Bar(x=recommend_score_counts.index, y=recommend_score_counts.values))
fig.update_layout(title='Distribution of Scores', xaxis_title='Score', yaxis_title='Frequency')
fig.show()

In [ ]:
########### Descriptive Analysis


